In [1]:
#-*- coding:utf-8 -*-

from bs4 import BeautifulSoup
import urllib.request
import csv
import datetime

In [2]:
alba_info = [""]*15

In [10]:
def alba_info_get(alba_url):
    alba_info = [""]*14
    
    try:

        with urllib.request.urlopen(alba_url) as url:  
            doc = url.read() #페이지에 있는 내용 가져와서 string 형식으로 저장해줌
        alba_doc = BeautifulSoup(doc, "html.parser")   
        s_alba = alba_doc.find('div', id = 'allcontent')
        s_company = s_alba.find('span', class_ = "companyName").text
        s_title = s_alba.h1.text

        alba_info[0] = alba_url
        alba_info[1] = s_company 
        alba_info[2] = s_title


        s_work_cond = s_alba.find('div', class_= "workCondition").find_all('tr')

        hired_type = ""
        category = ""

        for cond in s_work_cond:
            if cond.th.text == '고용형태':
                hired_type = cond.span.text
            elif cond.th.text == '업직종':
                category = cond.td.text.strip().split(",")

            #고용형태가 "연수생/교육생"이 아닌 경우에만 실행함
            alba_info[3] = [i.strip() for i in hired_type.split(",")] #고용형태
            alba_info[4] = category #업직종

            #연수생 교육생이 아닌경우 아래를 수행한다. 
            if "연수생/교육생" not in alba_info[3]:
                if cond.th.text == '근무기간':
                    hired_time = cond.span.text
                elif cond.th.text == "근무시간":
                    if '협의' not in cond.span.text:

                        workingtime = cond.find_all('span')[0].text.strip()
                        #restingtime = cond.find_all('span')[1].text.strip()
                        time = workingtime[:11].split("~")

                        # 야간, 새벽이 포함된 경우 'True'
                        if '(익일)' in workingtime:
                            night_work = True
                            workingtime.replace("(익일)", "")
                        else:
                            night_work = False
                            

                        #총 시간 구하는 부분. 
                        #하지만 알고보니 아래 json파일에 이미 사이트에서 제공되고 있었다....
                        timedelta = [time[i].split(":") for i in range(2)] #시작시간 , #끝나는시간
                        start_time = float(timedelta[0][0]) + float(int(timedelta[0][1])/60)
                        end_time = float(timedelta[1][0]) + float(int(timedelta[1][1])/60)                
                        total_time = end_time - start_time
                        #total time 
                        if night_work is True:
                            total_time += 24
                        alba_info[13] = workingtime
                        alba_info[5] = hired_time
                        alba_info[6] = total_time
                        alba_info[7] = night_work

            else:
                alba_info[5] = ""
                alba_info[6] = 0
                alba_info[7] = False

        s_work_cond = None

        #사이트 소스 중 아래쪽에 위치한 json파일 가져와서 dict로 읽기.   
        json_source = alba_doc.find_all("script", type = "text/javascript")#[28]

        for j in json_source:
            if 'viewData' in j.text:
                json_string = j.text.split(';')[0].replace("\r\n     ", " \"").replace("   ", "").replace(":", "\":").replace("var viewData = ", "").replace("\r\n ","").replace("\"}","}").replace("http\":", "http:")
                json_string = json_string.replace('false', "False").replace('true', 'True')
        dicta = eval(json_string)

        juso = dicta.get('gi_address', "")
        lat = dicta.get('map_lat', 0.0)
        long = dicta.get('map_lng', 0.0)
        map_x = dicta.get('map_x', 0.0)
        map_y = dicta.get('map_y', 0.0)
        realworkingtime = dicta.get('realWorkingTime', 0.0)

        alba_info[8] = juso #진짜 주소 
        alba_info[9] = lat #위도
        alba_info[10] = long #경도
        alba_info[11] = map_x #각각 좌표계 숫자. 
        alba_info[12] = map_y

        alba_info[6] = realworkingtime

        # [url, 기업, 알바명, 고용형태, 직업종류, 고용기간, 실근무시간, 야간근무여부, 상세주소, 위도, 경도, x좌표, y좌표]
        return alba_info
    
    except:
        return alba_info
    



In [11]:
alba_info_get("http://www.albamon.com/recruit/view/gi?AL_GI_No=67283100")

['http://www.albamon.com/recruit/view/gi?AL_GI_No=67283100',
 '코워커',
 '용인물류/구로역/당일지급/야간/장기 초보/즉시출근',
 ['알바'],
 ['포장·품질검사', '입출고·창고관리', '상하차·소화물 분류'],
 '1주일이하',
 '10',
 True,
 '용인시 처인구 원삼면 미평리 75',
 '37.1827812194824',
 '127.327621459961',
 '340494',
 '509212',
 '19:00~05:00(익일)']

# 여기서부터는 이전에 만들었던 알바 주소 목록에서 정보를 가져와서 같은 데이터프래임으로 만드는작업입니다

In [12]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

#원본파일:
original_csv = "alba_info_1215"

df = pd.read_csv(original_csv+ ".csv")

In [13]:
df['주소']

0        http://www.albamon.com//recruit/view/gi?AL_GI_No=67283499&amp;mj_stat=0&amp;optgf=
1        http://www.albamon.com//recruit/view/gi?AL_GI_No=67283498&amp;mj_stat=0&amp;optgf=
2        http://www.albamon.com//recruit/view/gi?AL_GI_No=67283497&amp;mj_stat=0&amp;optgf=
3        http://www.albamon.com//recruit/view/gi?AL_GI_No=67281131&amp;mj_stat=0&amp;optgf=
4        http://www.albamon.com//recruit/view/gi?AL_GI_No=67275043&amp;mj_stat=0&amp;optgf=
5        http://www.albamon.com//recruit/view/gi?AL_GI_No=67285310&amp;mj_stat=0&amp;optgf=
6        http://www.albamon.com//recruit/view/gi?AL_GI_No=67280818&amp;mj_stat=0&amp;optgf=
7        http://www.albamon.com//recruit/view/gi?AL_GI_No=67278982&amp;mj_stat=0&amp;optgf=
8        http://www.albamon.com//recruit/view/gi?AL_GI_No=67278971&amp;mj_stat=0&amp;optgf=
9        http://www.albamon.com//recruit/view/gi?AL_GI_No=67272673&amp;mj_stat=0&amp;optgf=
10       http://www.albamon.com//recruit/view/gi?AL_GI_No=67026051&amp;mj_stat=0

In [14]:
df.head()

,알바명,알바설명,지역,지역세부,급여종류,금액,주소
0,송파 장지 서울복합물류,"내일가능#당일지급#152,300원#초보가능#하루가능#야간풀",서울,광진구,일급,152300,http://www.albamon.com//recruit/view/gi?AL_GI_No=67283499&amp;mj_stat=0&amp;optgf=
1,송파 장지 서울복합물류,"내일가능#당일지급#152,300원#초보가능#하루가능#야간풀",서울,강북구,일급,152300,http://www.albamon.com//recruit/view/gi?AL_GI_No=67283498&amp;mj_stat=0&amp;optgf=
2,송파 장지 서울복합물류,"내일가능#당일지급#152,300원#초보가능#하루가능#야간풀",서울,송파구,일급,152300,http://www.albamon.com//recruit/view/gi?AL_GI_No=67283497&amp;mj_stat=0&amp;optgf=
3,레고코리아,[레고/단기] 이마트 경산점 레고 판촉(일급 8만),대구,전체,일급,80000,http://www.albamon.com//recruit/view/gi?AL_GI_No=67281131&amp;mj_stat=0&amp;optgf=
4,굿프랜드로지스틱스,당일지급/10만원/주5일근무/한달단기/물류센터 상품 분류,경기,성남시,일급,100000,http://www.albamon.com//recruit/view/gi?AL_GI_No=67275043&amp;mj_stat=0&amp;optgf=


In [15]:
# colname = [url주소, 기업, 알바명, 고용형태, 직업종류, 고용기간, 실근무시간, 야간근무여부, 상세주소, 위도, 경도, x좌표, y좌표]
import time

f = open(original_csv + "_specific.csv", 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
wr.writerow(['url주소', '기업', '알바명', '고용형태', '직업종류', '고용기간' ,'근무시간', '실근무시간', '야간근무여부', '상세주소', '위도', '경도', 'x좌표', 'y좌표'])

for i in range(len(df)):
    url = df['주소'][i]
    info_list = alba_info_get(url)
    print(url)
    wr.writerow([url, info_list[1], info_list[2], info_list[3], info_list[4], info_list[5], info_list[13], info_list[6], info_list[7], info_list[8], info_list[9], info_list[10], info_list[11], info_list[12]]) 

    time.sleep(0.5)

f.close()

http://www.albamon.com//recruit/view/gi?AL_GI_No=67283499&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67283498&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67283497&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67281131&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67275043&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67285310&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67280818&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67278982&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67278971&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67272673&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67026051&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257271&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67273092&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67273091&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67283260&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67274757&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67273089&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67283256&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67208902&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67208967&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67256279&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67254736&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67276573&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=66960900&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67228407&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67268388&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67267859&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67267386&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67266548&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67266419&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67266058&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67265704&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67265441&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67265222&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67208279&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67208230&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67275686&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67275685&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67275684&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67275683&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67275682&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67261518&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67277717&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67271307&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67208920&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67281365&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67263042&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67272755&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67279582&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67266424&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67198886&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67277109&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67266167&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67274526&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67239673&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67279082&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67277084&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67283525&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67275081&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67283514&amp;mj_stat=0&amp;optgf=
http

...

http://www.albamon.com//recruit/view/gi?AL_GI_No=67257737&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257733&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257727&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257724&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257721&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257719&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257718&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257717&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257716&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257715&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257714&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67257713&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67256027&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255982&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67250136&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255847&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255828&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255755&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255798&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255789&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255783&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255775&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255770&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67255769&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67252926&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67209621&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67209620&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67209619&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67248553&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67209618&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67209617&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67209113&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67206302&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67206270&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67194475&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67220640&amp;mj_stat=0&amp;optgf=
http

http://www.albamon.com//recruit/view/gi?AL_GI_No=67166787&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67196632&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67166786&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67196631&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67166785&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67166784&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67196630&amp;mj_stat=0&amp;optgf=
http://www.albamon.com//recruit/view/gi?AL_GI_No=67196629&amp;mj_stat=0&amp;optgf=


KeyboardInterrupt: 

In [36]:
df.head(25)

,알바명,알바설명,지역,지역세부,급여종류,금액,시간,주소,모집시간대,근로시간,야간,직업카테고리,위도,경도,실주소
0,(재)우체국물류지원단 안양물류사업소,안양우편물류센터 월요일용직 근무자 모집,경기,안양시,시급,8800,시간협의,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
1,애플트리(단기알바/정말 쉬운작업/초보환영,당일지급/마켓형 물류창고/파트타임(4.5시간)/실지급6만,경기,남양주,일급,90000,15:30~00:30,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
2,쿠팡로지스틱스서비스(유),0230분-0830근무)쿠팡_헬퍼_하루가능_셔틀_단기_장기_동,서울,강동구,시급,9000,시간협의,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
3,+해외배송 물류 선별작업,+[시급9.000원/당일지급/12.13일 이틀알바] 해외부품 선,인천,미추홀,시급,9000,09:00~18:00,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
4,"㈜알파모델에이전시(KBS,SBS,MBC,TvN 등)","[주급/초보] 드라마 영화 CF (보조출연),(연기자)성인,청",서울,금천구,일급,120000,시간협의,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
5,야간/곤지암 물류센터,12-15만원 당일지급] 곤지암 물류센터,서울,양천구,일급,152388,19:00~08:30,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
6,㈜에스코리아,"[일4H 4만원/롯데마트 잠실,송파점]롯데제과 生生빵상회",서울,송파구,시급,10000,16:00~21:00,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
7,야간/곤지암 물류센터,12-15만원 당일지급] 곤지암 물류센터,서울,도봉구,일급,152388,19:00~08:30,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
8,㈜에스코리아,"[일4H 4만원/롯데마트 구로,금천점]롯데제과 生生빵상회",서울,구로구,시급,10000,16:00~21:00,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,
9,야간/곤지암 물류센터,12-15만원 당일지급] 곤지암 물류센터,경기,성남시,일급,152388,19:00~08:30,http://www.albamon.com//recruit/view/gi?AL_GI_...,,,,,,,


In [ ]:
import csv
 
f = open('alba_info_1212.csv', 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
wr.writerow(['알바명', '알바설명', '지역', '지역세부', '급여종류', '금액', '시간', '주소'])

a = alba_list
for i in range(len(a)):
    wr.writerow([a[i][0], a[i][1], a[i][2][0], a[i][2][1], a[i][3],  a[i][4], a[i][5], a[i][6]]) 

    

f.close()